### Setup Instructions

If you haven't done so, please follow the [setup instructions](./nlp-tutorial-setup) to prepare your environment.  This tutorial will be referencing two pieces of code:


1.  [Notebook](https://github.com/outerbounds/tutorials/blob/main/nlp/nlp-5.ipynb) for this lesson.
2.  [predflow.py](https://github.com/outerbounds/tutorials/blob/main/nlp/predflow.py)

#### What You Will Learn
At the end of this lesson, you will:
    
* Learn how to retrieve your model for use in downstream systems with the client API.
* Learn how you retrieve your model in a separate flow for predictions.

In Episode 5, you saw how we trained a model and tagged the model if it passed certain tests to indicate that it was ready for downstream processes.  In this lesson, we show you how you can retrieve this model in other Flows, but also outside of flows with the client API.

In addition to manipulating tags as seen the previous lesson, the Metaflow [client API](https://docs.metaflow.org/api/client) allows you to access data from past runs.  For example, this is how you can retrieve a model tagged as a `deployment candidate` outside Metaflow:

In [ ]:
from metaflow import Flow

def get_latest_successful_run(flow_nm, tag):
    "Gets the latest successfull run for a flow with a specific tag."
    for r in Flow(flow_nm).runs(tag):
        if r.successful: return r

The above code allows you to retrieve runs for flows matching `flow_nm` and filter them according to whether or not they are tagged with `tag`. Finally, we check if the run is successfull with the `successful` property. 

After retrieving the model's data with the client API, we can load the model like this:

In [ ]:
#hide_output
#notest
from model import NbowModel

run = get_latest_successful_run('NLPFlow', 'deployment_candidate')
model = NbowModel.from_dict(run.data.model_dict)

Now that we have retrieved the model using the tag we can use it to make predictions:

In [ ]:
#notest
import pandas as pd

predict_df = pd.read_parquet('predict.parquet')
preds = model.predict(predict_df['review'])
preds

array([[0.99930894],
       [0.9928788 ],
       [0.99954474],
       ...,
       [0.9996772 ],
       [0.9996848 ],
       [0.23608246]], dtype=float32)

You can write these predictions to a parquet file like so:

In [ ]:
import pyarrow as pa
pa_tbl = pa.table({"data": preds.squeeze()})
pa.parquet.write_table(pa_tbl, "sentiment_predictions.parquet")

With the client API, you can retrieve your artifacts in whatever downstream application you want, or even just use the API for ad-hoc testing.

We can utilize the client api to also retrieve model artifacts within a flow!

 In this flow, we will perform the following steps:

1. Get the latest deployment candidate using the Metaflow API in the `start` step.  Recall that the name of our previous flow is `NLPFlow`.
2. Make predictions with our deployment candidate on a new dataset and write that to a parquet file in the `end` step

In [ ]:
%%writefile predflow.py

from metaflow import FlowSpec, step, Flow, current

class NLPredictionFlow(FlowSpec):
    
    def get_latest_successful_run(self, flow_nm, tag):
        "Gets the latest successfull run for a flow with a specific tag."
        for r in Flow(flow_nm).runs(tag):
            if r.successful: return r
        
    @step
    def start(self):
        "Get the latest deployment candidate that is from a successfull run"
        self.deploy_run = self.get_latest_successful_run('NLPFlow', 'deployment_candidate')
        self.next(self.end)
    
    @step
    def end(self):
        "Make predictions"
        from model import NbowModel
        import pandas as pd
        import pyarrow as pa
        new_reviews = pd.read_parquet('predict.parquet')['review']
        
        # Make predictions
        model = NbowModel.from_dict(self.deploy_run.data.model_dict)
        predictions = model.predict(new_reviews)
        print(f'Writing predictions to parquet: {predictions.shape[0]:,} rows')
        pa_tbl = pa.table({"data": predictions.squeeze()})
        pa.parquet.write_table(pa_tbl, "sentiment_predictions.parquet")
        
if __name__ == '__main__':
    NLPredictionFlow()

Writing predflow.py


In [ ]:
#notest
! python predflow.py run

Metaflow 2.7.1 executing NLPredictionFlow for user:hamel
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-08-11 14:52:02.786 Workflow starting (run-id 1660254722779868):
2022-08-11 14:52:02.794 [1660254722779868/start/1 (pid 37189)] Task is starting.
2022-08-11 14:52:03.577 [1660254722779868/start/1 (pid 37189)] Task finished successfully.
2022-08-11 14:52:03.587 [1660254722779868/end/2 (pid 37193)] Task is starting.
2022-08-11 14:52:06.101 [1660254722779868/end/2 (pid 37193)] 2022-08-11 14:52:06.101443: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
2022-08-11 14:52:06.452 [1660254722779868/end/2 (pid 37193)] Writing predictions to parquet: 2,264 rows
2022-08-11 14:52:06.742 [1660254722779868/end/2 (pid 37193)] To enable them in other operations, rebuild TensorFlo

### Conclusion

Congratulations!  You have learned how to:

1. Create a baseline flow that reads data and computes a baseline.
2. How to use branching to perform steps in parallel.
3. Best practices for serializing and de-serializing models in Metaflow.
4. How to use tagging to evaluate and gate models for production.
5. How to retrieve your model both outside Metaflow and from another flow.

### Further Discussion

This is a very simple example that will also run on your laptop.  However, for production use cases you may want to use [@conda](https://docs.metaflow.org/metaflow/dependencies#managing-dependencies-with-conda-decorator) for dependency management, [@batch](https://docs.metaflow.org/v/r/metaflow/scaling#using-aws-batch) or [@kubernetes](https://docs.metaflow.org/metaflow/scaling-out-and-up/effortless-scaling-with-kubernetes) for remote execution, and [@schedule](https://docs.metaflow.org/going-to-production-with-metaflow/scheduling-metaflow-flows/scheduling-with-aws-step-functions#scheduling-a-flow) to schedule jobs to run periodically.  